In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [2]:
import tensorflow as tf

tf.get_logger().setLevel(tf._logging.ERROR)
print(tf.config.get_visible_devices())

from keras.layers import (
    Layer,
    MultiHeadAttention,
    Dense,
    LayerNormalization,
    Dropout,
    Embedding,
    Input,
    TextVectorization,
    Flatten,
    LeakyReLU,
)
from keras import Model, losses, Sequential, callbacks, activations, optimizers, utils
import tensorflow as tf
from typing import Literal
import numpy as np
import re
from tensorflow_addons.layers import InstanceNormalization
import string


class MaskedSparseCategoricalCrossentropy(losses.Loss):
    def __init__(self, from_logits: bool = True, pad_value: int = 0, **kwargs):
        super().__init__(**kwargs)
        self.pad_value = pad_value
        self.loss = losses.SparseCategoricalCrossentropy(from_logits, reduction="none")

    def call(self, y_true: tf.Tensor, y_pred: tf.Tensor):
        loss = self.loss(y_true, y_pred)
        mask = tf.cast(y_true != self.pad_value, dtype=loss.dtype)
        loss *= mask
        loss = tf.reduce_sum(loss) / tf.reduce_sum(mask)
        return loss


class TransformerBlock(Layer):
    def __init__(
        self, embed_dim: int, num_heads: int, ff_dim: int, rate: float = 0.2, **kwargs
    ):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.ff_dim = ff_dim
        self.rate = rate

        self.dropout1 = Dropout(self.rate)
        self.dropout2 = Dropout(self.rate)
        # self.layernorm1 = LayerNormalization(epsilon=1e-6, center=True, scale=True)
        # self.layernorm2 = LayerNormalization(epsilon=1e-6, center=True, scale=True)

        self.layernorm1 = InstanceNormalization() # Loss dropped from 4.5 to 2.0
        self.layernorm2 = InstanceNormalization()

        self.mha = MultiHeadAttention(self.num_heads, self.embed_dim)
        self.ffn = Sequential(
            [
                Dense(self.ff_dim, activation="relu"),
                Dense(self.embed_dim),
            ]
        )

    def attention_mask(
        self, batch_size: int, n_dest: int, n_src: int, dtype: tf.DType
    ) -> tf.Tensor:
        i = tf.expand_dims(tf.range(n_dest), axis=-1)
        j = tf.range(n_src)
        mask = tf.cast(i >= j - n_src + n_dest, dtype)
        mask = tf.reshape(mask, [1, n_dest, n_src])
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
        )
        return tf.tile(mask, mult)

    def call(self, inputs: tf.Tensor) -> tf.Tensor:
        batch_size, seq_len = tf.shape(inputs)[0], tf.shape(inputs)[1]
        mask = self.attention_mask(batch_size, seq_len, seq_len, tf.bool)
        attention_output = self.mha(inputs, inputs, attention_mask=mask)
        attention_output = self.dropout1(attention_output)
        out = self.layernorm1(inputs + attention_output)
        ffn_out = self.ffn(out)
        ffn_out = self.dropout2(ffn_out)
        norm = self.layernorm2(out + ffn_out)
        return norm

    def get_config(self) -> dict:
        config = {
            "embed_dim": self.embed_dim,
            "ff_dim": self.ff_dim,
            "num_heads": self.num_heads,
            "rate": self.rate,
        }

        return config


class TokenAndPositionEmbedding(Layer):
    def __init__(self, max_len: int, vocab_size: int, embed_dim: int):
        super().__init__()
        self.max_len = max_len
        self.embed_dim = embed_dim
        self.vocab_size = vocab_size

        self.positions = tf.range(start=0, limit=self.max_len, delta=1)
        self.embedding_token = Embedding(
            input_dim=self.vocab_size, output_dim=self.embed_dim, mask_zero=True
        )
        self.embedding_position = Embedding(
            input_dim=self.max_len, output_dim=self.embed_dim
        )

    def call(self, x: tf.Tensor) -> tf.Tensor:
        positions = self.embedding_position(self.positions)
        x = self.embedding_token(x)
        _sum = x + positions
        return _sum

    def get_config(self) -> dict:
        config = {
            "max_len": self.max_len,
            "embed_dim": self.embed_dim,
            "vocab_size": self.vocab_size,
        }
        return config


class SaveModel(tf.keras.callbacks.Callback):
    def __init__(self, path: str, **kwargs):
        super().__init__(**kwargs)
        self.path = path
    
    def on_epoch_end(self, epoch, logs=None):
        self.model.save(self.path, save_format="tf")


class TextGenerator(callbacks.Callback):
    def __init__(
        self,
        seed_text: str,
        next_words: int,
        max_sequence_len: int,
        vectorize_layer: TextVectorization,
        top_k=10,
        print_every=1,
        model=None,
    ):
        self.seed_text = seed_text
        self.next_words = next_words
        self.max_sequence_len = max_sequence_len
        self.vectorize_layer = vectorize_layer
        if model is not None:
            self.model: Model = model
        self.print_every = print_every
        self.k = top_k

        vocab = vectorize_layer.get_vocabulary()
        self.int2word = {i: word for i, word in enumerate(vocab)}
        self.word2int = dict(zip(self.int2word.values(), self.int2word.keys()))

    def preprocess(self, content: str) -> str:
        to_left: str = r" A-Za-ząćęłńóśźż\-.,?!:;()\n"
        content = re.sub(f"[^{to_left}]+", "", content).lower()
        content = re.sub(f"([{string.punctuation}])", r" \1", content)
        content = re.sub("\n+", " \n ", content)
        content = re.sub(" +", " ", content)
        return content

    def sample_from(self, logits: np.ndarray) -> np.ndarray:
        indices = logits.argpartition(-self.k)[-self.k :].astype("int32")
        logits = logits[indices]

        preds = activations.softmax(tf.expand_dims(logits, 0))
        preds = np.array(preds[0]).astype("float32")
        return np.random.choice(indices, p=preds)

    def generate_text(self) -> str:
        start_tokens = self.preprocess(self.seed_text).split(" ")
        tokens_generated = []
        while len(tokens_generated) <= self.next_words:
            start_tokens = start_tokens[-self.max_sequence_len :]

            x = []
            for tok in start_tokens:
                if tok in self.word2int.keys():
                    x.append(self.word2int[tok])
            x = utils.pad_sequences(
                np.array(x)[np.newaxis], maxlen=self.max_sequence_len, padding="post"
            )

            y = self.model.predict_on_batch(x)[0]
            idx = min(len(start_tokens) - 1, self.max_sequence_len - 1)

            sample_token = self.sample_from(y[idx] if len(y.shape) == 2 else y)
            tokens_generated.append(sample_token)
            start_tokens.append(self.int2word[sample_token])

        token_to_word = []
        for tok in tokens_generated:
            try:
                word = self.int2word[tok]
                token_to_word.append(word)
            except:
                token_to_word.append("")
        txt = self.seed_text + " " + " ".join(token_to_word)
        txt = re.sub(r"\s([" + f"${string.punctuation}" + r"](?:\s|$))", r"\1", txt)
        return txt

    def on_epoch_begin(self, epoch: int, logs=None):
        if (epoch + 1) % self.print_every != 0:
            return
        txt = self.generate_text()
        print(f"Epoch: {epoch}; Generated text:\n{txt}\n")

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


/home/adrian/anaconda3/envs/tf_311/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
import tensorflow as tf
from keras.layers import TextVectorization
from glob import glob
from typing import List, Literal
import string
import re
import numpy as np
from keras import utils

fnames = list(glob("../texts/books-raw/*")) + list(glob("../texts/bajki-extend/*"))
# fnames = list(glob("../texts/bajki-extend/*"))


def create_dataset(
    filenames: List[str], output_sequence_length: int = 50, max_tokens: int = 50_000
):
    to_left: str = r" A-Za-ząćęłńóśźż\-.,?!:;()\n"

    def clear_dataset(input_str: str) -> str:
        lower = tf.strings.lower(input_str)
        filtered = tf.strings.regex_replace(lower, f"[^{to_left}]+", "")
        stripped = tf.strings.regex_replace(
            filtered, f"([{string.punctuation}])", r" \1"
        )
        newline = tf.strings.regex_replace(stripped, "\n+", " \n ")
        whitespace = tf.strings.regex_replace(newline, " +", " ")
        return whitespace

    ds: tf.data.Dataset = tf.data.TextLineDataset(filenames)
    ds = ds.map(lambda x: clear_dataset(x))
    ds = ds.filter(lambda x: tf.strings.length(x) > 2)
    ds = ds.batch(512)
    ds = ds.prefetch(tf.data.AUTOTUNE)

    vectorize_layer = TextVectorization(
        standardize=None,
        max_tokens=max_tokens,
        output_mode="int",
        output_sequence_length=output_sequence_length,
    )
    vectorize_layer.adapt(ds)
    vocab = vectorize_layer.get_vocabulary()
    _word2int = {word: i for i, word in enumerate(vocab)}
    _int2word = dict(zip(_word2int.values(), _word2int.keys()))

    def data_generator(data: List[int], max_len: int):
        while True:
            x, y = [], []
            for _ in range(1024):
                # _max_len = max_len
                _max_len = np.random.randint(5, max_len + 1)
                index = np.random.randint(len(data) - _max_len - 1)
                x.append(data[index : index + _max_len])
                y.append(data[index + 1 : index + _max_len + 1])

            x = utils.pad_sequences(x, max_len, padding="post")
            y = utils.pad_sequences(y, max_len, padding="post")
            for _x, _y in zip(x, y):
                yield _x, _y

    data = ""
    for filename in filenames:
        with open(filename, "r", encoding="utf8") as f:
            content = re.sub(f"[^{to_left}]+", "", f.read()).lower()
            content = re.sub(f"([{string.punctuation}])", r" \1", content)
            content = re.sub("\n+", " \n ", content)
            content = re.sub(" +", " ", content)
            data += content

    data_int = []
    for key in data.split(" "):
        if key in _word2int.keys():
            data_int.append(_word2int[key])

    ds = tf.data.Dataset.from_generator(
        lambda: data_generator(data_int, output_sequence_length),
        output_signature=(
            tf.TensorSpec(shape=(output_sequence_length,), dtype=tf.int32),
            tf.TensorSpec(shape=(output_sequence_length,), dtype=tf.int32),
        ),
    )
    ds = ds.prefetch(tf.data.AUTOTUNE)

    return ds, _word2int, _int2word, vectorize_layer

def create_model(max_sequence_len: int, total_words: int) -> Model:
    embed_dim = 256
    num_heads = 4
    ff_dim = 256
    inputs = Input(shape=(max_sequence_len,))
    x = TokenAndPositionEmbedding(max_sequence_len, total_words, embed_dim)(inputs)
    x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)
    x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)
    x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)
    outputs = Dense(total_words)(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss=MaskedSparseCategoricalCrossentropy(from_logits=True),
        optimizer=optimizers.Adam(1e-3),
    )
    return model

output_sequence_length = 30
max_tokens = 50_000
epochs = 30
ds, word2int, int2word, vectorize_layer = create_dataset(
    fnames, output_sequence_length, max_tokens
)

seed = "Kiedy księżyc wzeszedł, wziął Jaś siostrzyczkę za rękę i poszedł z nią śladem kamyków, które błyszczały w świetle księżycowym jak nowiutkie pieniążki i pokazywały im drogę. Szli całą noc, a gdy dzień nastał, doszli do domu ojca.".lower()

model = create_model(output_sequence_length, max_tokens)
model.summary()
model.fit(
    ds.batch(128).prefetch(tf.data.AUTOTUNE),
    verbose=1,
    epochs=epochs,
    steps_per_epoch=1500,
    callbacks=[
        TextGenerator(seed, 60, output_sequence_length, vectorize_layer, 5),
        SaveModel("../transformer_models/model_best_2.tf")
        # tf.keras.callbacks.ModelCheckpoint(
        #     "../transformer_models/model_best_2.h5",
        #     monitor="loss",
        #     save_best_only=True,
        #     save_weights_only=False,
        # ),
    ],
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 30)]              0         
                                                                 
 token_and_position_embeddin  (None, 30, 256)          12807680  
 g (TokenAndPositionEmbeddin                                     
 g)                                                              
                                                                 
 transformer_block (Transfor  (None, 30, 256)          1184512   
 merBlock)                                                       
                                                                 
 transformer_block_1 (Transf  (None, 30, 256)          1184512   
 ormerBlock)                                                     
                                                                 
 transformer_block_2 (Transf  (None, 30, 256)          118451

1500/1500 [==============================] - 124s 82ms/step - loss: 5.1041
Epoch: 1; Generated text:
kiedy księżyc wzeszedł, wziął jaś siostrzyczkę za rękę i poszedł z nią śladem kamyków, które błyszczały w świetle księżycowym jak nowiutkie pieniążki i pokazywały im drogę. szli całą noc, a gdy dzień nastał, doszli do domu ojca.- , dość jakieś wszystkie aż u u się się się. nie. nie. nie, to. ale że. ,, że, z jego, na pani coraz u tej nim się. .. nie się nie się. , że, że, , na jego, że na pani w tej na jeszcze

Epoch 2/30
1500/1500 [==============================] - ETA: 0s - loss: 3.4599

1500/1500 [==============================] - 123s 82ms/step - loss: 3.4599
Epoch: 2; Generated text:
kiedy księżyc wzeszedł, wziął jaś siostrzyczkę za rękę i poszedł z nią śladem kamyków, które błyszczały w świetle księżycowym jak nowiutkie pieniążki i pokazywały im drogę. szli całą noc, a gdy dzień nastał, doszli do domu ojca. książę ksiądz dam małe jedynym niezmiernie wprost temu- jednego pod i na, nie się, i, .. . w. w. to! . nie mieli, w tej mną na, z i za swój swoje, , który, na tym, i w, .. . gdyby z i, jak się

Epoch 3/30
1500/1500 [==============================] - ETA: 0s - loss: 2.6265

1500/1500 [==============================] - 124s 83ms/step - loss: 2.6265
Epoch: 3; Generated text:
kiedy księżyc wzeszedł, wziął jaś siostrzyczkę za rękę i poszedł z nią śladem kamyków, które błyszczały w świetle księżycowym jak nowiutkie pieniążki i pokazywały im drogę. szli całą noc, a gdy dzień nastał, doszli do domu ojca.- choroba nieprzyjaciel straszne straszny sto rynku drugich stosunek pamiętał zdobył się, nie, . na tym to na tym się, . w nocy. : nie pan on nie bóg nie się się. , w ręku, nie w razie w tej dnia, w, jak się się w tej chwili, że nie z powodu

Epoch 4/30
1500/1500 [==============================] - ETA: 0s - loss: 2.1684

1500/1500 [==============================] - 123s 82ms/step - loss: 2.1684
Epoch: 4; Generated text:
kiedy księżyc wzeszedł, wziął jaś siostrzyczkę za rękę i poszedł z nią śladem kamyków, które błyszczały w świetle księżycowym jak nowiutkie pieniążki i pokazywały im drogę. szli całą noc, a gdy dzień nastał, doszli do domu ojca.- rzędzian cierpi kandyd obchodzi chrzestnego lutego szkoły całą przy całą, na tym, że. . to. się w to z. . to go nie się go być siła, że mu mu mnie mu im mu się ku niemu: że i i i w której. a po francusku. po czym z ciebie,

Epoch 5/30
1500/1500 [==============================] - ETA: 0s - loss: 1.8471

...

1500/1500 [==============================] - 123s 82ms/step - loss: 1.0696
Epoch: 11; Generated text:
kiedy księżyc wzeszedł, wziął jaś siostrzyczkę za rękę i poszedł z nią śladem kamyków, które błyszczały w świetle księżycowym jak nowiutkie pieniążki i pokazywały im drogę. szli całą noc, a gdy dzień nastał, doszli do domu ojca. powiadasz myślę obiecałem- wielokrotnie, wyglądzie tropy wnętrze ale tej. . gdy. kto! co nie. nie. nie, a. nie jest, nie jest, co go, że, nie jest na łac. , że nawet. jeżeli. dopiero? .. nie ma. . czy to nie powinien mi

Epoch 12/30
1500/1500 [==============================] - ETA: 0s - loss: 1.0066

1500/1500 [==============================] - 123s 82ms/step - loss: 1.0066
Epoch: 12; Generated text:
kiedy księżyc wzeszedł, wziął jaś siostrzyczkę za rękę i poszedł z nią śladem kamyków, które błyszczały w świetle księżycowym jak nowiutkie pieniążki i pokazywały im drogę. szli całą noc, a gdy dzień nastał, doszli do domu ojca. daruj dziwną! - rozumiem, - zarumienione i i wojownika. .. , co nie ma się w głowie, gdzie, co się i jak jak, nie z tego drogi; jak nie z ust, jak sam. .. nie jest jak na ziemię, bo i od siebie i nie po niej nie będzie

Epoch 13/30
1500/1500 [==============================] - ETA: 0s - loss: 0.9544

1500/1500 [==============================] - 122s 81ms/step - loss: 0.9544
Epoch: 13; Generated text:
kiedy księżyc wzeszedł, wziął jaś siostrzyczkę za rękę i poszedł z nią śladem kamyków, które błyszczały w świetle księżycowym jak nowiutkie pieniążki i pokazywały im drogę. szli całą noc, a gdy dzień nastał, doszli do domu ojca. cha obiecałem- nastanie zadrżał, grudki, ; i i na, w ten, nie i w oczy, jak, jak nie jest z jego. nie ma z tego. . o sobie. a o tym za chwilę, a to o to tak jednak tak na znak; a w tym to po powrocie,

Epoch 14/30
1500/1500 [==============================] - ETA: 0s - loss: 0.9115

1500/1500 [==============================] - 123s 82ms/step - loss: 0.9115
Epoch: 14; Generated text:
kiedy księżyc wzeszedł, wziął jaś siostrzyczkę za rękę i poszedł z nią śladem kamyków, które błyszczały w świetle księżycowym jak nowiutkie pieniążki i pokazywały im drogę. szli całą noc, a gdy dzień nastał, doszli do domu ojca.! myślę! zabit- -- krętych w. stanowczego, to pan to. w ręku i to, że, a to nie chciał się na siebie. gdy z tego rzeczy, które z jego rzeczy z jego. na chwilę do teatru. po czym, a w tym nie jest z i na cmentarz.

Epoch 15/30
 120/1500 [=>............................] - ETA: 1:50 - loss: 0.8891

KeyboardInterrupt: 

In [ ]:
fnames2 = list(glob("../texts/bajki-extend/*"))
ds2, _, _, _ = create_dataset(fnames2)

model.fit(
    ds.batch(128),
    verbose=1,
    epochs=10,
    steps_per_epoch=1500,
    callbacks=[
        TextGenerator(seed, 60, output_sequence_length, vectorize_layer, 5),
        tf.keras.callbacks.ModelCheckpoint(
            "../transformer_models/model_best_3.h5",
            monitor="loss",
            save_best_only=True,
            save_weights_only=False,
        ),
    ],
)

In [ ]:
seed = "spójrz , jak pięknie kwitną dokoła kwiatki , dlaczego nie patrzysz na nie ?".lower()
for i in [1, 2, 4, 8, 10, 20]:
    txt = TextGenerator(seed, 60, output_sequence_length, vectorize_layer, i, model=model).generate_text(),
    print(txt)
    # with open(f'../generated_texts/transformer/text_{i}', 'w') as f:
    #     f.write(f'Seed: {seed}\n')
    #     f.write(txt)